In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

dataset = pd.read_csv('titanic.csv')
dataset

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [2]:
train_data, test_data = train_test_split(dataset, train_size = 0.7, stratify = dataset['Survived'])

grouping_dataset = dataset.groupby(by=['Survived']).count()

grouping_train_dataset = train_data.groupby(by=['Survived']).count()

grouping_test_dataset = test_data.groupby(by=['Survived']).count()

print("Persentase training data set per class label")
print(grouping_train_dataset['PassengerId'] / grouping_dataset['PassengerId'])

print("\nPersentase test data set per class label")
print(grouping_test_dataset['PassengerId'] / grouping_dataset['PassengerId'])

train_label = pd.DataFrame(train_data, columns=['Survived'])
test_label = pd.DataFrame(test_data, columns=['Survived'])

train_data['Sex'] = train_data['Sex'].replace(['male', 'female'], [1, 0])
test_data['Sex'] = test_data['Sex'].replace(['male', 'female'], [1, 0])

train_data = train_data.fillna(train_data.groupby('Survived').transform('mean'))
train_data = pd.DataFrame(train_data, columns=['Sex', 'Age', 'Pclass', 'Fare'])

test_data = pd.DataFrame(test_data, columns=['Sex', 'Age', 'Pclass', 'Fare'])
test_data = test_data.dropna()

kNN = KNeighborsClassifier(n_neighbors = 3, weights='distance')
kNN.fit(train_data, train_label)
class_result = kNN.predict(test_data)
rows, cols = test_data.shape
precision_ratio = kNN.score(test_data, test_label.iloc[0:rows])
error_ratio = 1 - precision_ratio
print('\nHoldout Error Rate = {}'.format(error_ratio))

Persentase training data set per class label
Survived
0    0.699454
1    0.698830
Name: PassengerId, dtype: float64

Persentase test data set per class label
Survived
0    0.300546
1    0.301170
Name: PassengerId, dtype: float64

Holdout Error Rate = 0.4608294930875576


<ipython-input-2-9e58b26946ab>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Sex'] = train_data['Sex'].replace(['male', 'female'], [1, 0])
<ipython-input-2-9e58b26946ab>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['Sex'] = test_data['Sex'].replace(['male', 'female'], [1, 0])
C:\Users\shiroe\AppData\Roaming\Python\Python38\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the 

In [3]:
train_label

,Survived
456,0
319,1
640,0
817,0
620,0
...,...
50,0
235,0
322,1
553,1


In [4]:
test_label

,Survived
854,0
352,0
446,1
606,0
202,0
...,...
166,1
243,0
774,1
140,0


In [5]:
from sklearn.model_selection import KFold
import numpy as np

data = pd.DataFrame(dataset, columns = ['Sex', 'Age', 'Pclass', 'Fare'])
label = pd.DataFrame(dataset, columns = ['Survived'])

n_split = 10

kFold = KFold(n_splits = n_split, shuffle = False)

In [7]:
final_error_rate = 0

for i, (train_index, test_index) in enumerate(kFold.split(data, label), 1):
    temp_index = np.copy(train_index)
    temp_test_index = np.copy(test_index)
    train_data, test_data = data.loc[temp_index], data.loc[temp_test_index]
    train_label, test_label = label.loc[temp_index], label.loc[temp_test_index]
    
    train_data = train_data.fillna(dataset.loc[temp_index].groupby('Survived').transform('mean'))
    min_max = train_data.agg([min, max])
    min_max_train_data = train_data.agg([min, max])
    
    train_data['Sex'] = train_data['Sex'].replace(['male', 'female'], [1, 0])
    train_data = (train_data - train_data.min()) / (train_data.max() - train_data.min())

    test_data['Sex'] = test_data['Sex'].replace(['male', 'female'], [1, 0])
    test_data = (test_data - test_data.min()) / (test_data.max() - test_data.min())
    test_data = test_data.dropna()
    
    kNN = KNeighborsClassifier(n_neighbors = 3, weights='distance')
    kNN.fit(train_data, train_label)
    class_result = kNN.predict(test_data)
    rows, cols = test_data.shape
    precision_ratio = kNN.score(test_data, test_label.iloc[0:rows])
    error_ratio = 1 - precision_ratio

    print("{}.\tK - Fold Error Rate = {} ".format(i, error_ratio))

C:\Users\shiroe\AppData\Roaming\Python\Python38\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\shiroe\AppData\Roaming\Python\Python38\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\shiroe\AppData\Roaming\Python\Python38\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\shiroe\AppData\Roaming\Python\Python38\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was

1.	K - Fold Error Rate = 0.536231884057971 
2.	K - Fold Error Rate = 0.36 
3.	K - Fold Error Rate = 0.36111111111111116 
4.	K - Fold Error Rate = 0.46478873239436624 
5.	K - Fold Error Rate = 0.5633802816901409 


C:\Users\shiroe\AppData\Roaming\Python\Python38\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\shiroe\AppData\Roaming\Python\Python38\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\shiroe\AppData\Roaming\Python\Python38\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\shiroe\AppData\Roaming\Python\Python38\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was

6.	K - Fold Error Rate = 0.3484848484848485 
7.	K - Fold Error Rate = 0.4347826086956522 
8.	K - Fold Error Rate = 0.34246575342465757 
9.	K - Fold Error Rate = 0.5616438356164384 
10.	K - Fold Error Rate = 0.42666666666666664 


In [8]:
from sklearn.model_selection import LeaveOneOut

data = pd.DataFrame(dataset, columns=['Sex', 'Age', 'Pclass', 'Fare'])
label = pd.DataFrame(dataset, columns=['Survived'])

loo = LeaveOneOut()

In [13]:
final_error_rate = 0

for train_index, test_index in loo.split(data, label):
    temp_index = np.copy(train_index)
    temp_test_index = np.copy(test_index)
    train_data, test_data = data.loc[temp_index], data.loc[temp_test_index]
    train_label, test_label = label.loc[temp_index], label.loc[temp_test_index]

    train_data = train_data.fillna(dataset.loc[temp_index].groupby('Survived').transform('mean'))
    min_max = train_data.agg([min, max])
    min_max_train_data = train_data.agg([min, max])
    
    train_data['Sex'] = train_data['Sex'].replace(['male', 'female'], [1, 0])
    train_data = (train_data - train_data.min()) / (train_data.max() - train_data.min())

    test_data['Sex'] = test_data['Sex'].replace(['male', 'female'], [1, 0])
    test_data = (test_data - test_data.min()) / (test_data.max() - test_data.min())
    
    if not test_data.isna().any(axis=1).bool():  
        kNN = KNeighborsClassifier(n_neighbors = 3, weights='distance')
        kNN.fit(train_data, train_label)
        class_result = kNN.predict(test_data)
        rows, cols = test_data.shape
        precision_ratio = kNN.score(test_data, test_label.iloc[0:rows])
        error_ratio = 1 - precision_ratio

        final_error_rate += error_ratio
    
print("LOO Error Rate = {} ".format(final_error_rate / data.shape[0]))

LOO Error Rate = 0.0 


In [ ]:
train_label

In [ ]:
min_max_train_data = train_data.agg([min, max])
train_data['Sex'] = train_data['Sex'].replace(['male', 'female'], [1, 0])
train_data = (train_data - train_data.min()) / (train_data.max() - train_data.min())
train_data['Sex'] = train_data['Sex'].replace([1, 0], ['male', 'female'])
print('Min Max Train Data')
print(min_max_train_data)
print('\n------------------------------------------------------\n')
print('Train Data')
print(train_data)

In [ ]:
test_data = test_data[['Sex', 'Age', 'Pclass', 'Fare']].dropna()
new_min_max_test_data = train_data.agg([min, max])
test_data['Sex'] = test_data['Sex'].replace(['male', 'female'], [1, 0])
test_data = (test_data - test_data.min()) / (test_data.max() - test_data.min())
test_data['Sex'] = test_data['Sex'].replace([1, 0], ['male', 'female'])
print('Min Max Test Data')
print(new_min_max_test_data)
print('\n------------------------------------------------------\n')
print('Test Data')
print(test_data)

In [ ]:
k = 3
kNN = KNeighborsClassifier(n_neighbors=k, weights='distance')
train_data['Sex'] = train_data['Sex'].replace(['male', 'female'], [1, 0])
test_data['Sex'] = test_data['Sex'].replace(['male', 'female'], [1, 0])
kNN.fit(train_data, train_label)
class_result = kNN.predict(test_data)
rows, cols = test_data.shape
precision_ratio = kNN.score(test_data, test_label.iloc[0:rows])
error_ratio = 1 - precision_ratio
print('\n------------------------------------------------------\n')
print('K {} --> {}'.format(k, error_ratio))
print('\n------------------------------------------------------\n')